# Imports


In [27]:
import random
import gradio as gr
import copy

cross_prob = 0.65
mut_prob = 0.05

# Flower



In [28]:
class Flower:
    def __init__(self, sizeCenter , colorCenter , colorPetals , colorStem , numPetals): 
        self.sizeCenter = sizeCenter
        self.colorCenter = colorCenter
        self.colorPetals = colorPetals
        self.colorStem = colorStem
        self.numPetals = numPetals
        self.fitness = 1

    @property
    def attribute(self):
        return [
            self.sizeCenter,
            *self.colorCenter,
            *self.colorPetals,
            *self.colorStem,
            self.numPetals
        ]
        # self.attribute = [
        #     self.sizeCenter, 
        #     *self.colorCenter,
        #     *self.colorPetals,
        #     *self.colorStem,
        #     self.numPetals,
        #     # self.fitness
        # ]

    def set_fitness(self,new_fit):
        self.fitness = new_fit
        
    def display(self):
        print(f"------------- Flower Details: -------------")
        print(f"Center Size: {self.sizeCenter}")
        print(f"Center Color: {self.colorCenter}")
        print(f"Petals Color: {self.colorPetals}")
        print(f"Stem Color: {self.colorStem}")
        print(f"Number of Petals: {self.numPetals}")
        print(f"Fitness value: {self.fitness}")
        print(f"array of Attributes: {self.attribute}")



# Population


In [29]:
# Assume sizeCenter is 8 bits 
# Assume numPetals is 8 bits
# we may change the colorCenter, colorPetals, colorStem to be 24 bits each (8 bits for R, 8 bits for G, 8 bits for B)

def createPopulation():
    population = []  
    for i in range(8):
        sizeCenter = random.randint(10, 30)
        colorCenter = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorPetals = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorStem = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        numPetals = random.randint(0, 7)
        flower = Flower(sizeCenter, colorCenter, colorPetals, colorStem, numPetals)
        population.append(flower)
    return population



In [30]:
def convertDecimalToBinary(n):
    return bin(n).replace("0b", "").zfill(8)

def convertDecimalToBinary3(n):
    return bin(n).replace("0b", "").zfill(3)

def convertDecimalToBinary5(n):
    return bin(n).replace("0b", "").zfill(5)

print(convertDecimalToBinary(7))


00000111


In [31]:
def displayBinaryRepresentation(flower):
    sizeCenterBinary = convertDecimalToBinary5(flower.sizeCenter) # 5 bits to encapsulate 30 ( in the mail 10 to 30)
    colorCenterBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorCenter]) 
    colorPetalsBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorPetals])
    colorStemBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorStem])
    numPetalsBinary = convertDecimalToBinary3(flower.numPetals)
    
    chromosome = sizeCenterBinary + colorCenterBinary + colorPetalsBinary + colorStemBinary + numPetalsBinary
    return chromosome

def Chromosomes(population):
    result = []
    for i, flower in enumerate(population):
        result.append(displayBinaryRepresentation(flower))
    return result


  

In [32]:
# test displays binary

# for flower in population:
#     flower.display()
#     print("Binary Representation: ", displayBinaryRepresentation(flower))
#     print()


# Chromosomes = Chromosomes(population)
# for i, chromosome in enumerate(Chromosomes):
#     print(f"Flower {i+1} Chromosome: {chromosome}")   

In [33]:
def convertBinaryToDecimal(chromosome):
    decimal = 0
    for i in range(len(chromosome)):
        decimal += int(chromosome[i]) * (2 ** (len(chromosome) - 1 - i))
    return decimal
   
    # return int(chromosome, 2) does the same 


In [34]:
# print(convertBinaryToDecimal("00110001"))

In [35]:
def repair(min_value, max_value,x_value):
    return max(min_value, min(max_value, x_value))

# min 0
# max 1000

# num > 1000 2000
# num < 0  -1000

# Selection


In [36]:
import random

# we can change this slightly to return the reordered list
#  with the probabilities of each flower for visualization

def roulette_selection(population):
    
    # remaining = population.copy()
    new_population = []

    for _ in range(len(population)):
        total_fitness = sum(f.fitness for f in population)

        cumaltive = []
        acc = 0 
        for f in population:
            acc += f.fitness / total_fitness
            cumaltive.append(acc)
        
        r = random.random()

        for i, f in enumerate(population):
            if r <= cumaltive[i]:
                new_population.append(f)
                # remaining.pop(i)
                break

    return new_population

# Crossover


In [37]:
def update_flower(flower1, newList):
    flower1.sizeCenter = newList[0]
    flower1.colorCenter = (newList[1] , newList[2] , newList[3])
    flower1.colorPetals = (newList[4] , newList[5] , newList[6])
    flower1.colorStem = (newList[7] , newList[8] , newList[9])
    flower1.numPetals = newList[10]
    flower1.attribute = newList
    



In [38]:
def crossover( flower1, flower2):
    crossPos = random.randint(1, len(flower1.attribute)-1-1)

    # Perform crossover
    if(random.random() < cross_prob):
        print(crossPos)
        offspring1 = flower1.attribute[:crossPos] + flower2.attribute[crossPos:]
        offspring2 = flower2.attribute[:crossPos] + flower1.attribute[crossPos:]

        # Create new flower objects for offspring
        offspring3 = Flower(
            offspring1[0],
            (offspring1[1], offspring1[2], offspring1[3]),
            (offspring1[4], offspring1[5], offspring1[6]),
            (offspring1[7], offspring1[8], offspring1[9]),
            offspring1[10]
        )
        offspring4 = Flower(
            offspring2[0],
            (offspring2[1], offspring2[2], offspring2[3]),
            (offspring2[4], offspring2[5], offspring2[6]),
            (offspring2[7], offspring2[8], offspring2[9]),
            offspring2[10]
        )
    else:
        offspring3 = flower1
        offspring4 = flower2

    # update_flower(flower1, offspring1)
    # update_flower(flower2, offspring2)
    
    return offspring3, offspring4

## cross_prop = 0.65
## rabdam.randam()< 0.65
    ## child 1 = 
    ## child 2 =
## else
    ## child 1 = parent 1
    ## child 2 = parent 2
## reurn child 1 , child 2


## mutation
## mut_prob = 0.05
    ## flower1 : [12,34,255,108]
    ## center_size = random.randint(10,30)

## generation

## pop = evolve(generation)

## generation = pop

In [39]:
# #test crossover
# import copy

# copy.deepcopy(population[2])  # deep copy (safer)
# flower3 = copy.deepcopy(population[1])
# flower4 = copy.deepcopy(population[2])

# flower3.display()
# flower4.display()
# off1, off2 = crossover(flower3,flower4)
# flower3.display()
# flower4.display()
# population[1].display()
# population[2].display()

# Mutation

In [40]:
def mutation(population):
    for flower in population:
        if random.random() < mut_prob:
            flower.sizeCenter = random.randint(10, 30)

        r, g, b = flower.colorCenter
        if random.random() < mut_prob:
            r = random.randint(0, 255)
        if random.random() < mut_prob:
            g = random.randint(0, 255)
        if random.random() < mut_prob:
            b = random.randint(0, 255)

        flower.colorCenter = (r, g, b)

        r, g, b = flower.colorPetals
        if random.random() < mut_prob:
            r = random.randint(0, 255)
        if random.random() < mut_prob:
            g = random.randint(0, 255)
        if random.random() < mut_prob:
            b = random.randint(0, 255)

        flower.colorPetals = (r, g, b)

        r, g, b = flower.colorStem
        if random.random() < mut_prob:
            r = random.randint(0, 255)
        if random.random() < mut_prob:
            g = random.randint(0, 255)
        if random.random() < mut_prob:
            b = random.randint(0, 255)

        flower.colorStem = (r, g, b)

        if random.random() < mut_prob:
            flower.numPetals = random.randint(0, 7)

    return population
            

In [41]:
# def mutation(population):
#     # i want for each bit to be mutated by 5%
#     # mutation_rate = 0.05
#     i = 0
#     j = 0
#     k = 0
#     for flower in population:
#         k += 1
#         chromosome = displayBinaryRepresentation(flower)
#         new_chromosome = ""
#         for bit in chromosome:
#             if random.random() < mut_prob:
#                 i = i + 1
#                 # Flip the bit
#                 if bit == '0':
#                     new_bit = '1'
#                 else:
#                     new_bit = '0'
#                 new_chromosome += new_bit
#             else:
#                 new_chromosome += bit
#         print("Old chromosome flower ", k, ": ", chromosome)
#         print("New chromosome flower ", k, ": ", new_chromosome)
#         print("Mutations in flower: ", i-j)
#         j = i
#         # ------------------- My code ends here this is copilot suggestion -------------------

#         # Update flower attributes based on new chromosome
#         flower.sizeCenter = repair(10, 30, convertBinaryToDecimal(new_chromosome[0:5]))
#         flower.colorCenter = (
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[5:13])),
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[13:21])),
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[21:29]))
#         )
#         flower.colorPetals = (
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[29:37])),
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[37:45])),
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[45:53]))
#         )
#         flower.colorStem = (
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[53:61])),
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[61:69])),
#             repair(0, 255, convertBinaryToDecimal(new_chromosome[69:77]))
#         )
#         flower.numPetals = repair(0, 7, convertBinaryToDecimal(new_chromosome[77:80]))
#         flower.attribute = [flower.sizeCenter, *flower.colorCenter, *flower.colorPetals, *flower.colorStem, flower.numPetals]
#     print("Total mutations: ", i)
# # mutation(population)

# Evolution ( all functions working together)

In [42]:
population = createPopulation()

In [43]:
for f in population:
    print(f.attribute)

[25, 25, 110, 15, 197, 14, 22, 163, 124, 165, 0]
[10, 63, 211, 215, 236, 169, 121, 162, 231, 73, 1]
[30, 40, 158, 4, 95, 95, 201, 98, 202, 102, 2]
[22, 4, 201, 115, 230, 142, 106, 100, 19, 144, 6]
[29, 74, 19, 40, 56, 61, 75, 218, 20, 8, 5]
[18, 67, 68, 223, 187, 86, 94, 145, 98, 144, 4]
[21, 55, 28, 120, 220, 80, 205, 79, 150, 233, 1]
[18, 159, 212, 173, 50, 31, 98, 120, 234, 114, 0]


In [44]:
def evolution(population):
    # print population at the beginning
    print("population before roullette selection")
    for flower in population:
        print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

    # Sort population by fitness (highest first)
    sorted_population = sorted(population, key=lambda f: f.fitness, reverse=True)

    # Print sorted population
    print("Population (After Sorting by Fitness)")
    for flower in sorted_population:
        print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

    population = roulette_selection(population)
    # print population after selection (ordered for crossover)
    print("population after selection: ")
    for flower in population:
        print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

    # print((flower.attribute , flower.fitness)  for flower in population)

    for i in range( 0, len(population) , 2):
        #print population[i] and of i+1
        print("flowers before crossover (if crossover even occurs)")
        print(f"Attributes: {population[i].attribute}, Fitness: {population[i].fitness}")
        print(f"Attributes: {population[i+1].attribute}, Fitness: {population[i+1].fitness}")
        offspring1, offspring2 = crossover(population[i] , population[i+1])
        population[i] = offspring1
        population[i+1] = offspring2

        print("flowers after crossover (if crossover even occurs)")
        print(f"Attributes: {population[i].attribute}, Fitness: {population[i].fitness}")
        print(f"Attributes: {population[i+1].attribute}, Fitness: {population[i+1].fitness}")
        # i+=1
        # print(f"crossover {i}")
        # print after crossover (just re print as crossover changes in the object) 
    # return fitness to 1
    for flower in population:
        flower.fitness = 1
    print("total population after crossover: ")
    # print((flower.attribute , flower.fitness)  for flower in population)
    for flower in population:
        print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

    #print population before mutation
    population = mutation(population)
    #print populatin after mutation

    print("population after mutation: ")
    for flower in population:
        print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

    print("------------------ This is the end of 1 evolution ------------")

    # print the population after everything 
    return population


In [45]:
# # population = createPopulation()
# for flower in population:
#         print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

# population = evolution(population)

# for flower in population:
#         print(f"Attributes: {flower.attribute}, Fitness: {flower.fitness}")

# GUI

In [46]:
# import gradio as gr

# def rgb_str(rgb):
#     return f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})"

# def flower_frame_html(index, flower):
#     return f"""
#     <div class="flower-frame">
#         <canvas id="flowerCanvas{index}" width="200" height="200"></canvas>
#         <div id="timer{index}" class="timer">Timer: 0</div>
#     </div>
#     <script>
#     (function() {{
#         const canvas = document.getElementById("flowerCanvas{index}");
#         const ctx = canvas.getContext("2d");
#         const timerDisplay = document.getElementById("timer{index}");
#         const centerSize = {flower.sizeCenter / 20:.2f};
#         const centerColor = "{rgb_str(flower.colorCenter)}";
#         const petalColor = "{rgb_str(flower.colorPetals)}";
#         const stemColor = "{rgb_str(flower.colorStem)}";
#         const numPetals = {flower.numPetals};

#         function drawFlower() {{
#             const centerX = 100, centerY = 100;
#             const petalRadius = 15;
#             const flowerRadius = 40;

#             // Draw stem
#             ctx.beginPath();
#             ctx.strokeStyle = stemColor;
#             ctx.lineWidth = 6;
#             ctx.moveTo(100, 200);
#             ctx.lineTo(100, 130);
#             ctx.stroke();

#             // Draw petals
#             ctx.fillStyle = petalColor;
#             for (let i = 0; i < numPetals; i++) {{
#                 const angle = (2 * Math.PI / numPetals) * i;
#                 const x = centerX + flowerRadius * Math.cos(angle);
#                 const y = centerY + flowerRadius * Math.sin(angle);
#                 ctx.beginPath();
#                 ctx.arc(x, y, petalRadius, 0, 2 * Math.PI);
#                 ctx.fill();
#             }}

#             // Draw center
#             ctx.beginPath();
#             ctx.fillStyle = centerColor;
#             ctx.arc(centerX, centerY, centerSize, 0, 2 * Math.PI);
#             ctx.fill();
#         }}

#         drawFlower();

#         let time = 0;
#         let timerInterval = null;

#         function startTimer() {{
#             if (!timerInterval) {{
#                 timerInterval = setInterval(() => {{
#                     time += 1;
#                     timerDisplay.textContent = "Timer: " + time;
#                     window.flowerTimers[{index}] = time;
#                 }}, 100);
#             }}
#         }}

#         function stopTimer() {{
#             clearInterval(timerInterval);
#             timerInterval = null;
#         }}

#         canvas.addEventListener("mouseenter", startTimer);
#         canvas.addEventListener("mouseleave", stopTimer);
#     }})();
#     </script>
#     """

# def build_html(population):
#     html = """
#     <!doctype html>
#     <html>
#     <head>
#         <meta charset="utf-8" />
#         <style>
#             body {
#                 font-family: Arial, sans-serif;
#                 display: flex;
#                 flex-wrap: wrap;
#                 justify-content: center;
#                 align-items: center;
#                 gap: 20px;
#                 padding: 20px;
#                 margin: 0;
#             }
#             .flower-frame {
#                 display: flex;
#                 flex-direction: column;
#                 align-items: center;
#                 justify-content: center;
#                 border: 2px solid #4CAF50;
#                 border-radius: 10px;
#                 background: #f9f9f9;
#                 width: 220px;
#                 height: 260px;
#                 box-shadow: 0 0 5px rgba(0,0,0,0.1);
#                 transition: box-shadow 0.3s ease;
#             }
#             .flower-frame:hover {
#                 box-shadow: 0 0 15px rgba(76, 175, 80, 0.6);
#             }
#             canvas {
#                 border-radius: 10px;
#                 background: #f9f9f9;
#                 cursor: pointer;
#             }
#             .timer {
#                 margin-top: 8px;
#                 font-size: 16px;
#                 font-weight: bold;
#                 color: #333;
#             }
#         </style>
#     </head>
#     <body>
#         <script>window.flowerTimers = new Array(8).fill(0);</script>
#     """
#     for i, f in enumerate(population):
#         html += flower_frame_html(i, f)
#     html += "</body></html>"
#     return html

# # ---- Python-side logic ----
# def update_fitness_from_timers(timers_json):
#     if not timers_json:
#         return "⚠️ No timers received from JS."
    
#     try:
#         import json
#         timers = json.loads(timers_json)
#         timers = [int(t) for t in timers]
        
#         # Update fitness for each flower
#         for i, t in enumerate(timers):
#             population[i].set_fitness(t)
        
#         # Create detailed report showing fitness values
#         report = "✅ **Fitness Updated Successfully!**\n\n"
#         report += "| Flower # | Timer Value | Fitness Value |\n"
#         report += "|----------|-------------|---------------|\n"
#         for i, flower in enumerate(population):
#             report += f"| {i} | {timers[i]} | {flower.fitness} |\n"
        
#         return report
#     except Exception as e:
#         return f"❌ Error: {str(e)}"

# def show_current_fitness():
#     """Display current fitness values of all flowers"""
#     report = "**Current Fitness Values:**\n\n"
#     report += "| Flower # | Fitness |\n"
#     report += "|----------|----------|\n"
#     for i, flower in enumerate(population):
#         report += f"| {i} | {flower.fitness} |\n"
#     return report

# # ---- Gradio UI ----
# with gr.Blocks(theme="default") as demo:
#     gr.Markdown("## 🌸 Hover over each flower to increase its timer")
    
#     iframe_html = f'<iframe id="flowerFrame" srcdoc="{build_html(population).replace(chr(34), "&quot;")}" style="width:100%; height:850px; border:none;"></iframe>'
#     gr.HTML(value=iframe_html)
    
#     with gr.Row():
#         get_timers_btn = gr.Button("💪 Update Fitness from Hover Time", variant="primary")
#         check_fitness_btn = gr.Button("🔍 Check Current Fitness", variant="secondary")
    
#     timers_input = gr.Textbox(visible=False)  # Hidden textbox to receive JS data
#     result_display = gr.Markdown("")
    
#     get_timers_btn.click(
#         fn=None,
#         inputs=None,
#         outputs=timers_input,
#         js="""
#         () => {
#             const iframe = document.getElementById("flowerFrame");
#             if (!iframe) {
#                 console.error("❌ iframe not found");
#                 return "[]";
#             }
#             const timers = iframe.contentWindow.window.flowerTimers;
#             console.log("🌸 Timers from iframe:", timers);
#             return JSON.stringify(timers || []);
#         }
#         """
#     ).then(
#         fn=update_fitness_from_timers,
#         inputs=timers_input,
#         outputs=result_display
#     )
    
#     check_fitness_btn.click(
#         fn=show_current_fitness,
#         inputs=None,
#         outputs=result_display
#     )

# demo.launch()

In [47]:
# demo.close()
global population

# sawy = population

In [66]:
from flask import Flask, request, jsonify, render_template_string
import uuid
session_id = str(uuid.uuid4())

app = Flask(__name__)

def rgb_str(rgb):
    return f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})"

def flower_frame_html(index, flower):
    print(f"Drawing flower {index}: center={flower.colorCenter}, petals={flower.colorPetals}, stem={flower.colorStem}, numPetals={flower.numPetals}")
    return f"""
    <div class="flower-frame">
        <canvas id="flowerCanvas{index}" width="200" height="200"></canvas>
        <div id="timer{index}" class="timer">Timer: 1</div>
    </div>
    <script>
    (function() {{
        const canvas = document.getElementById("flowerCanvas{index}");
        const ctx = canvas.getContext("2d");
        const timerDisplay = document.getElementById("timer{index}");
        const centerSize = {flower.sizeCenter / 1:.2f};
        const centerColor = "{rgb_str(flower.colorCenter)}";
        const petalColor = "{rgb_str(flower.colorPetals)}";
        const stemColor = "{rgb_str(flower.colorStem)}";
        const numPetals = {flower.numPetals};

        function drawFlower() {{
            const centerX = 100, centerY = 100;
            const petalRadius = 15;
            const flowerRadius = 40;

            ctx.beginPath();
            ctx.strokeStyle = stemColor;
            ctx.lineWidth = 6;
            ctx.moveTo(100, 200);
            ctx.lineTo(100, 130);
            ctx.stroke();

            ctx.fillStyle = petalColor;
            for (let i = 0; i < numPetals; i++) {{
                const angle = (2 * Math.PI / numPetals) * i;
                const x = centerX + flowerRadius * Math.cos(angle);
                const y = centerY + flowerRadius * Math.sin(angle);
                ctx.beginPath();
                ctx.arc(x, y, petalRadius, 0, 2 * Math.PI);
                ctx.fill();
            }}

            ctx.beginPath();
            ctx.fillStyle = centerColor;
            ctx.arc(centerX, centerY, centerSize, 0, 2 * Math.PI);
            ctx.fill();
        }}
        drawFlower();


        let time = 1;
        let timerInterval = null;
        window.flowerTimers[{index}] = 1;

        function startTimer() {{
            if (!timerInterval) {{
                timerInterval = setInterval(() => {{
                    time += 1;
                    timerDisplay.textContent = "Timer: " + time;
                    window.flowerTimers[{index}] = time;
                }}, 100);
            }}
        }}

        function stopTimer() {{
            clearInterval(timerInterval);
            timerInterval = null;
        }}

        canvas.addEventListener("mouseenter", startTimer);
        canvas.addEventListener("mouseleave", stopTimer);
    }})();
    </script>
    """

def build_html(population, session_id):
    html = """
    <!doctype html>
    <html>
    <head>
        <meta charset="utf-8" />
        <title>🌸 Flower Hover Game</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                display: flex;
                flex-wrap: wrap;
                justify-content: center;
                align-items: center;
                gap: 20px;
                padding: 20px;
                margin: 0;
            }
            .flower-frame {
                display: flex;
                flex-direction: column;
                align-items: center;
                justify-content: center;
                border: 2px solid #4CAF50;
                border-radius: 10px;
                background: #f9f9f9;
                width: 220px;
                height: 260px;
                box-shadow: 0 0 5px rgba(0,0,0,0.1);
                transition: box-shadow 0.3s ease;
            }
            .flower-frame:hover {
                box-shadow: 0 0 15px rgba(76, 175, 80, 0.6);
            }
            canvas {
                border-radius: 10px;
                background: #f9f9f9;
                cursor: pointer;
            }
            .timer {
                margin-top: 8px;
                font-size: 16px;
                font-weight: bold;
                color: #333;
            }
            button {
                position: fixed;
                top: 20px;
                right: 20px;
                padding: 10px 15px;
                font-size: 16px;
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 6px;
                cursor: pointer;
            }
        </style>
    </head>
    <body>
        <button id="sendButton">Send Timers to Flask 0</button>
        <script>window.flowerTimers = new Array(8).fill(1);</script>
    """
    for i, f in enumerate(population):
        html += flower_frame_html(i, f)
    html += """
    {% raw %}
    <script>
    if (!localStorage.getItem("sessionId") || localStorage.getItem("sessionId") !== '""" + session_id + """') {
        localStorage.clear();
        localStorage.setItem("sessionId", '""" + session_id + """');
    }
    let i = parseInt(localStorage.getItem("clickCount")) || 0;
    const button = document.getElementById('sendButton');
    button.textContent = `Send Timers to Flask ${i}`;
    async function sendTimers() {
        i++;
        button.disabled = true;
        button.textContent = 'Sending...';
        try {
            const res = await fetch('/update_fitness', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({timers: window.flowerTimers})
            });
            const data = await res.json();
            alert(data.message);
            localStorage.setItem("clickCount", i);
            window.location.reload();
        } catch (err) {
            alert('❌ Error sending timers');
        } finally {
            button.disabled = false;
            button.textContent = `Send Timers to Flask ${i}`;
        }
    }
    document.getElementById('sendButton').addEventListener('click', sendTimers);
    </script>
    {% endraw %}
    </body></html>
    """
    return html



@app.route("/update_fitness", methods=["POST"])
def update_fitness():
    global population
    timers = request.json.get("timers", [])
    for i, t in enumerate(timers):
        population[i].set_fitness(max(1, int(t)))
     
    population = evolution(population)
    return jsonify({"message": "✅ Fitness updated!", "timers": timers})

@app.route("/")
def index():
    global population, session_id
    return render_template_string(build_html(population, session_id))


app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.16:5000
Press CTRL+C to quit
192.168.1.16 - - [22/Oct/2025 14:37:37] "GET / HTTP/1.1" 200 -


Drawing flower 0: center=(4, 209, 98), petals=(156, 90, 195), stem=(239, 185, 41), numPetals=7
Drawing flower 1: center=(4, 209, 98), petals=(101, 90, 8), stem=(249, 150, 41), numPetals=7
Drawing flower 2: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 41), numPetals=7
Drawing flower 3: center=(77, 209, 98), petals=(156, 2, 34), stem=(199, 150, 80), numPetals=7
Drawing flower 4: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 41), numPetals=7
Drawing flower 5: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 28), numPetals=7
Drawing flower 6: center=(4, 209, 98), petals=(156, 2, 54), stem=(199, 189, 80), numPetals=7
Drawing flower 7: center=(4, 209, 98), petals=(156, 85, 161), stem=(249, 185, 41), numPetals=7


192.168.1.16 - - [22/Oct/2025 14:37:41] "POST /update_fitness HTTP/1.1" 200 -


population before roullette selection
Attributes: [26, 4, 209, 98, 156, 90, 195, 239, 185, 41, 7], Fitness: 3
Attributes: [26, 4, 209, 98, 101, 90, 8, 249, 150, 41, 7], Fitness: 2
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fitness: 8
Attributes: [26, 77, 209, 98, 156, 2, 34, 199, 150, 80, 7], Fitness: 3
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fitness: 5
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 28, 7], Fitness: 3
Attributes: [26, 4, 209, 98, 156, 2, 54, 199, 189, 80, 7], Fitness: 1
Attributes: [26, 4, 209, 98, 156, 85, 161, 249, 185, 41, 7], Fitness: 2
Population (After Sorting by Fitness)
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fitness: 8
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fitness: 5
Attributes: [26, 4, 209, 98, 156, 90, 195, 239, 185, 41, 7], Fitness: 3
Attributes: [26, 77, 209, 98, 156, 2, 34, 199, 150, 80, 7], Fitness: 3
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 28, 7], Fitne

...

Drawing flower 0: center=(4, 56, 98), petals=(156, 194, 195), stem=(239, 185, 41), numPetals=7
Drawing flower 1: center=(4, 209, 98), petals=(156, 2, 54), stem=(199, 189, 80), numPetals=7
Drawing flower 2: center=(4, 209, 98), petals=(156, 2, 34), stem=(199, 150, 91), numPetals=7
Drawing flower 3: center=(77, 209, 98), petals=(156, 154, 161), stem=(249, 185, 41), numPetals=4
Drawing flower 4: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 41), numPetals=7
Drawing flower 5: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 28), numPetals=7
Drawing flower 6: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 28), numPetals=7
Drawing flower 7: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 41), numPetals=7


192.168.1.16 - - [22/Oct/2025 14:37:45] "POST /update_fitness HTTP/1.1" 200 -


population before roullette selection
Attributes: [26, 4, 56, 98, 156, 194, 195, 239, 185, 41, 7], Fitness: 4
Attributes: [17, 4, 209, 98, 156, 2, 54, 199, 189, 80, 7], Fitness: 2
Attributes: [26, 4, 209, 98, 156, 2, 34, 199, 150, 91, 7], Fitness: 3
Attributes: [26, 77, 209, 98, 156, 154, 161, 249, 185, 41, 4], Fitness: 9
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fitness: 1
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 28, 7], Fitness: 1
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 28, 7], Fitness: 1
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fitness: 1
Population (After Sorting by Fitness)
Attributes: [26, 77, 209, 98, 156, 154, 161, 249, 185, 41, 4], Fitness: 9
Attributes: [26, 4, 56, 98, 156, 194, 195, 239, 185, 41, 7], Fitness: 4
Attributes: [26, 4, 209, 98, 156, 2, 34, 199, 150, 91, 7], Fitness: 3
Attributes: [17, 4, 209, 98, 156, 2, 54, 199, 189, 80, 7], Fitness: 2
Attributes: [26, 4, 209, 98, 156, 90, 161, 249, 185, 41, 7], Fit

192.168.1.16 - - [22/Oct/2025 14:37:46] "GET / HTTP/1.1" 200 -


Drawing flower 0: center=(77, 209, 98), petals=(156, 154, 161), stem=(249, 185, 41), numPetals=4
Drawing flower 1: center=(77, 209, 98), petals=(156, 154, 161), stem=(249, 185, 41), numPetals=4
Drawing flower 2: center=(77, 209, 98), petals=(156, 154, 161), stem=(249, 185, 41), numPetals=4
Drawing flower 3: center=(4, 56, 98), petals=(156, 194, 195), stem=(239, 185, 41), numPetals=7
Drawing flower 4: center=(4, 209, 98), petals=(156, 90, 195), stem=(239, 185, 41), numPetals=7
Drawing flower 5: center=(4, 192, 98), petals=(156, 194, 161), stem=(249, 185, 41), numPetals=7
Drawing flower 6: center=(4, 209, 98), petals=(156, 2, 34), stem=(199, 150, 244), numPetals=7
Drawing flower 7: center=(4, 209, 98), petals=(156, 90, 161), stem=(249, 185, 41), numPetals=7


In [ ]:
for f in population:
    f.display()

------------- Flower Details: -------------
Center Size: 25
Center Color: (200, 102, 105)
Petals Color: (23, 219, 232)
Stem Color: (252, 41, 93)
Number of Petals: 4
Fitness value: 1
array of Attributes: [25, 200, 20, 133, 23, 219, 232, 252, 41, 93, 4]
------------- Flower Details: -------------
Center Size: 25
Center Color: (200, 102, 105)
Petals Color: (23, 219, 232)
Stem Color: (252, 41, 93)
Number of Petals: 4
Fitness value: 1
array of Attributes: [25, 200, 20, 133, 23, 219, 232, 252, 41, 93, 4]
------------- Flower Details: -------------
Center Size: 25
Center Color: (200, 20, 133)
Petals Color: (23, 219, 232)
Stem Color: (252, 41, 93)
Number of Petals: 4
Fitness value: 1
array of Attributes: [25, 200, 20, 133, 23, 219, 232, 252, 41, 93, 4]
------------- Flower Details: -------------
Center Size: 25
Center Color: (200, 20, 133)
Petals Color: (23, 219, 232)
Stem Color: (229, 41, 93)
Number of Petals: 4
Fitness value: 1
array of Attributes: [25, 200, 20, 133, 23, 219, 232, 252, 41, 9